## Pre-requisite: RAG101

## Background

We will address the context window issue we mentioned in RAG101. First, what is a context window? According to [Lark](https://www.larksuite.com/en_us/topics/ai-glossary/context-window-llms):

> A context window refers to a defined span of words within a text sequence that the Large Language Model (LLM) utilizes to extract contextual information.

The larger the context window, the more words/tokens the LLM can utilize to extract contextual information. As a result, the model with a larger context window can have a longer conversation.

Think of a scenario where the context you passed into your prompt template is a whole textbook. If the tokens in the textbook are more than your model's context window, then your model can only consider the most recent tokens up to their maximum limit. 

What if there is a way to make your model consider only the relevant portion of your textbook? This is where embeddings come in.

## Aim

Our aim in RAG103 is to build a system that uses embeddings to extract the most relevant part of the provided context to answer a query.